# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [2]:
# load dataframe from WeatherPy and display
filepath = "../WeatherPy/Output/weather_output.csv"
weather_data = pd.read_csv(filepath)
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,zaysan,47.4667,84.8667,71.44,38,4,4.99,kz,1627328043
1,ushuaia,-54.8000,-68.3000,37.06,87,75,8.05,ar,1627327869
2,ucluelet,48.9329,-125.5528,77.95,81,4,1.99,ca,1627327913
3,praxedis guerrero,31.3667,-106.0167,92.43,30,100,0.38,mx,1627328045
4,barrow,71.2906,-156.7887,46.42,99,90,9.22,us,1627327757
...,...,...,...,...,...,...,...,...,...
527,nicoya,10.1483,-85.4520,87.35,70,100,2.30,cr,1627328255
528,toora-khem,52.4675,96.1100,58.84,98,100,2.51,ru,1627328256
529,gasa,27.9037,89.7269,58.60,91,100,3.24,bt,1627328256
530,coihaique,-45.5752,-72.0662,44.55,53,20,6.91,cl,1627328256


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# create figure layout options in dictionary
figure_layout = {
    "width": "850px",
    "height": "500px",
    "border": "5px solid black",
    "padding": "1px"
}

# create variables for heatmap - to be weighted by humidity & max intensity will be max humidity
locations = weather_data[["Lat","Lng"]]
weight = weather_data["Humidity"]
max_humidity = weather_data["Humidity"].max()

# create and display figure with heatmap
fig = gmaps.figure(layout=figure_layout, zoom_level=1.7, center=(20,10))
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight, max_intensity=max_humidity, point_radius=15)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='5px solid black', height='500px', padding='1px', width='850px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# create ideal new dataframe containing only cities with max temp between 72F & 80F, 
# cloudiness less than 10%, and Humidity less than 50%
vacation_data = weather_data.loc[(weather_data["Max Temp"]>=72) & 
                                 (weather_data["Max Temp"]<=80) & 
                                 (weather_data["Cloudiness"]<10) &
                                 (weather_data["Humidity"]<50), :]

# display dataframe
vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
177,capao bonito,-24.0058,-48.3494,75.31,36,0,4.63,br,1627328112
309,yumen,40.2833,97.2000,75.36,22,7,11.56,cn,1627328166
352,piranshahr,36.7010,45.1413,76.30,25,2,4.03,ir,1627328183
431,cumra,37.5732,32.7745,77.14,35,0,9.48,tr,1627328215
448,hanna,51.6334,-111.9018,76.51,44,0,3.00,ca,1627328222
531,severnyy,50.6770,36.5532,73.44,43,0,6.71,ru,1627328257


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# select only needed rows from original dataframe
hotel_data = vacation_data.loc[:,["City","Lat","Lng","Country"]]
hotel_data["Hotel Name"] = None

# base url for Google Places API - Nearby Search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# set parameters to search for Hotel within 5000 meters of city coordinates
params = {
    "keyword": "hotel",
    "type": "lodging",
    "key": g_key,
    "radius": 5000,
    "location": ""
}

# perform successive API calls to collect hotel name
for index, row in hotel_data.iterrows():
    
    # insert city coordinates into parameters
    params["location"] = str(row[1]) +","+ str(row[2])
    
    # API call
    response = requests.get(base_url, params).json()
    
    # step into results part of dictionary
    hotel_results = response["results"]
    
    # if no error in finding hotel, print hotel name and add to dataframe
    try:
        print(f"{row['City']}, {row['Country'].upper()} hotel: {hotel_results[0]['name']}")
        hotel_data.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    # if error in finding hotel, print error statement
    except (KeyError, IndexError):
        print(f"No hotel found for this city within 5000 meters.")

# drop any rows where hotel is still null (not found)
hotel_data = hotel_data.dropna(how="any")
hotel_data

capao bonito,  BR hotel: Hotel Baguassu
No hotel found for this city within 5000 meters.
piranshahr,  IR hotel: Hotel Ronak
cumra,  TR hotel: ÇUMRA OTELİ
hanna,  CA hotel: Canalta Hotel Hanna
severnyy,  RU hotel: Hotel VOYAGE


,City,Lat,Lng,Country,Hotel Name
177,capao bonito,-24.0058,-48.3494,br,Hotel Baguassu
352,piranshahr,36.7010,45.1413,ir,Hotel Ronak
431,cumra,37.5732,32.7745,tr,ÇUMRA OTELİ
448,hanna,51.6334,-111.9018,ca,Canalta Hotel Hanna
531,severnyy,50.6770,36.5532,ru,Hotel VOYAGE


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
marker_locations = hotel_data[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='5px solid black', height='500px', padding='1px', width='850px'))